# Decorators
*Why use decorators instead of (e.g.) asserts*
1. Clutters functions with error-checking logic
2. If validation logic needs to change, many inline copies need to be found and updated 

## Anatomy of a Decorator

```python
def decorator(input_fn):
    def _decorate(*args, **kwargs):
        print('decorating')

        return input_fn(*args, **kwargs)

    return _decorate
```

In [7]:
def fndecorator(input_fn):
    def decorator():
        print('This is from decorator')

        return input_fn()

    return decorator 


@fndecorator
def new_fn():
    print('from original function')

In [8]:
new_fn()

This is from decorator
from original function


In [9]:
def manager_albany(*args):
    BLUE = '\033[94m'
    BOLD = '\33[5m'
    SELECT = '\33[7m'
    for arg in args:
        print(BLUE + BOLD + SELECT + str(arg))

In [16]:
def function_with_input(*args):
    for arg in args:
        print(arg)

def add_line_function(function_with_input):
    def add_line(*args):
        print('ADDED LINE')
        return function_with_input(*args)
    return add_line

In [17]:
@add_line_function
def fn(*args):
    for arg in args:
        print(arg)

In [18]:
fn('a', 'b', 'c')

ADDED LINE
a
b
c


In [23]:
def decorator(input_fn):
    def _decorate(*args, **kwargs):
        print('decorating')

        return input_fn(*args, **kwargs)

    return _decorate

In [24]:
@decorator
def fn(input_arg):
    print(input_arg)
    

In [26]:
def datefixer(fn):
    import datetime
    def decorator(*args):
        newargs = []
        for arg in args:
            if isinstance(arg, datetime.date):
                arg = arg.weekday(), arg.day, arg.month, arg.year
            newargs.append(arg)
        return fn(*newargs)
    return decorator


In [27]:
@datefixer
def set_holidays(*args):
    return args[0]

In [28]:
from datetime import datetime as dt

some_date = dt.strptime('2022-12-25', '%Y-%m-%d')
some_date

datetime.datetime(2022, 12, 25, 0, 0)

In [29]:
set_holidays(some_date)

(6, 25, 12, 2022)

In [202]:
import pandas as pd
from pyspark.sql import SparkSession

sc = SparkSession.builder.appName('decorators').getOrCreate()

data = pd.DataFrame({
    'american': ['06/07/2022'],
    'monthname': ['06/July/2022'],
    'julian': ['1997/310'],
    'inversejulian': ['310/1997'],

})

data

,american,monthname,julian,inversejulian
0,06/07/2022,06/July/2022,1997/310,310/1997


In [203]:
df = sc.createDataFrame(data)
df.show()

+----------+------------+--------+-------------+
|  american|   monthname|  julian|inversejulian|
+----------+------------+--------+-------------+
|06/07/2022|06/July/2022|1997/310|     310/1997|
+----------+------------+--------+-------------+



In [204]:
from pyspark.sql import functions as F 
df.withColumn('american', F.to_date('american', 'dd/MM/yyyy')).show()

+----------+------------+--------+-------------+
|  american|   monthname|  julian|inversejulian|
+----------+------------+--------+-------------+
|2022-07-06|06/July/2022|1997/310|     310/1997|
+----------+------------+--------+-------------+



In [205]:

df.withColumn('julian', F.to_date('julian', 'yyyy/DDD')).show()

+----------+------------+----------+-------------+
|  american|   monthname|    julian|inversejulian|
+----------+------------+----------+-------------+
|06/07/2022|06/July/2022|1997-11-06|     310/1997|
+----------+------------+----------+-------------+



In [206]:

df.withColumn('inversejulian', F.to_date('inversejulian', 'DDD/yyyy')).show()

+----------+------------+--------+-------------+
|  american|   monthname|  julian|inversejulian|
+----------+------------+--------+-------------+
|06/07/2022|06/July/2022|1997/310|   1997-11-06|
+----------+------------+--------+-------------+



In [207]:
df.withColumn('monthname', F.to_date('monthname', 'dd/LL/yyyy')).show()

+----------+---------+--------+-------------+
|  american|monthname|  julian|inversejulian|
+----------+---------+--------+-------------+
|06/07/2022|     null|1997/310|     310/1997|
+----------+---------+--------+-------------+



In [127]:
import functools


date_cols = {
    'american': ['american', 'american1'],
    'julian': ['julian']
}


def datefixer(dateconf):
    import pyspark
    def _datefixer(func):

        @functools.wraps(func)
        def wrapper(df, *args, **kwargs):
            df_dateconf = {}
            for key, values in dateconf.items():
                df_dateconf[key] = [i for i in df.columns if i in values]


            for dateformat in df_dateconf.keys():
                for datecolumn in df_dateconf[dateformat]:
                    print('converting', dateformat)
                    if dateformat == 'american':
                        df = df.withColumn(datecolumn, F.to_date(datecolumn, 'dd/MM/yyyy'))
                    if dateformat == 'julian':
                        df = df.withColumn(datecolumn, F.to_date(datecolumn, 'yyyy/DDD'))
            return func(df, *args, **kwargs)

        return wrapper

    return _datefixer

In [128]:
@datefixer(dateconf=date_cols)
def test(df):

    return df 

        

In [129]:
df.show()
test(df=df).show()

+----------+--------+-------------+
|  american|  julian|inversejulian|
+----------+--------+-------------+
|06/07/2022|1997/310|     310/1997|
+----------+--------+-------------+

converting american
converting julian
+----------+----------+-------------+
|  american|    julian|inversejulian|
+----------+----------+-------------+
|2022-07-06|1997-11-06|     310/1997|
+----------+----------+-------------+



In [154]:
import pyspark 
def fn(*args, **kwargs):
    for k, v in kwargs.items():
        if isinstance(v, pyspark.sql.DataFrame):
            kwargs[k] = kwargs[k].withColumn('new', F.lit(0))
    return kwargs 
    print(kwargs)


In [179]:
def datefixer(conf):
    import pyspark
    def _datefixer(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for k, v in kwargs.items():
                if isinstance(v, pyspark.sql.DataFrame):
                    print('found a dataframe')
                    kwargs[k] = kwargs[k].withColumn('col', F.lit(0))

            return func(*args, **kwargs)

        return wrapper

    return _datefixer
        

In [180]:
@datefixer(conf=None)
def process(df):
    return df

In [181]:
process(df=df).show()

found a dataframe
+----------+--------+-------------+---+
|  american|  julian|inversejulian|col|
+----------+--------+-------------+---+
|06/07/2022|1997/310|     310/1997|  0|
+----------+--------+-------------+---+

